In [ ]:
import pm4py
import sys
sys.path.insert(0, '../scripts/')
from utils import print_stats
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.algo.simulation.playout.petri_net import algorithm as simulator


log = pm4py.read_xes(
    '../data/logs/transaction_methods/one_year_land_proxy_txs.xes')

print_stats(log)

bpmn_graph = pm4py.read_bpmn(
    "../data/models/transaction_methods/one_year/one_year_land_proxy_IMf.bpmn")

net, im, fm = pm4py.convert_to_petri_net(bpmn_graph)

# net,im,fm = pm4py.read_pnml("../data/models/transaction_methods/full_land/full_land_proxy_heuristic_default.pnml")


**Playout of a Petri Net**: Takes as input a Petri net along with an initial marking, and returns a list of process executions that are allowed from the process model. Two types of playout:

- _Variants.BASIC_PLAYOUT_: A basic playout that accepts a Petri net along with an initial marking, and returns a specified number of process executions (repetitions may be possible).

In [ ]:

# optional. Specify number of traces and max trace length
basic_parameters = {simulator.Variants.BASIC_PLAYOUT.value.Parameters.NO_TRACES: 50,
                    simulator.Variants.BASIC_PLAYOUT.value.Parameters.MAX_TRACE_LENGTH: 7}

basic_playout_log = simulator.apply(
    net, im, variant=simulator.Variants.BASIC_PLAYOUT, parameters=basic_parameters)

xes_exporter.apply(basic_playout_log, f"../data/logs/pn_basic_simulation.xes")

- _Variants.EXTENSIVE_: A playout that accepts a Petri net along with an initial marking, and returns all the executions that are possible according to the model, up to a provided length of trace (may be computationally expensive).

In [ ]:
# optional. Specify max trace length
extensive_parameters = {simulator.Variants.EXTENSIVE.value.Parameters.MAX_TRACE_LENGTH: 7}
extensive_log = simulator.apply(net, im, variant=simulator.Variants.EXTENSIVE, parameters=extensive_parameters)

xes_exporter.apply(extensive_log, f"../data/logs/pn_extensive_simulation.xes")

**Extensive playout of a process tree**: doing an extensive playout of Petri Nets can be really expensive. Process Tree permits to obtain the entire language of an event log in a much easier way. However, we need to specify some parameters: max number of occurrences for a loop, max length of a trace to be returned, max number of traces to be returned.
Reasonable settings are: max number of traces to 100000, max length of traces by default are set to the minimum length of a trace accepted by a process tree, and the max number of loops is set to be the minimum length of a trace divided by two.


In [ ]:
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.algo.simulation.playout.process_tree import algorithm as tree_playout

tree = inductive_miner.apply_tree(
    log, variant=inductive_miner.Variants.IMf)

playout_variant = tree_playout.Variants.EXTENSIVE
param = tree_playout.Variants.EXTENSIVE.value.Parameters

simulated_log = tree_playout.apply(tree, variant=playout_variant,
                                   parameters={param.MAX_TRACE_LENGTH: 20, param.MAX_LIMIT_NUM_TRACES: 100000})

xes_exporter.apply(simulated_log, f"../data/logs/pt_extensive_simulation.xes")